In [1]:
import numpy as np
import tensorflow as tf
from my_utils import load_mnist_data
from matplotlib import pyplot as plt
from time import time

In [2]:
train_images, train_labels, test_images, test_labels = load_mnist_data()

In [3]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [4]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 28, 28, 1])

In [5]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
L3 = tf.layers.dense(tf.reshape(L2, [-1, 7 * 7 * 64]), units=256, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer())

In [8]:
y_ = tf.layers.dense(L3, units=10, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer())

In [9]:
learning_rate = 1e-3
loss_history = []
m = 60000
epoch = 5
mini_batch_size = 128

y = tf.placeholder(tf.float32, [None, 10])

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

start = time()
# <--
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    indices = np.arange(0, m)
    
    for i in range(epoch):
        np.random.shuffle(indices)
        xs = train_images[indices, :]
        ys = train_labels[indices, :]
        
        for j in range(m // mini_batch_size):
            batch_xs = xs[j * mini_batch_size : (j+1) * mini_batch_size, :].reshape(-1, 28, 28, 1)
            batch_ys = ys[j * mini_batch_size : (j+1) * mini_batch_size, :]
            loss_val, _ = sess.run([loss, train_step], feed_dict={x: batch_xs, y: batch_ys})
            loss_history.append(loss_val)
    # -->    
    finish = time()
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_val = sess.run(accuracy, feed_dict={x: test_images.reshape(-1, 28, 28, 1), y: test_labels})
    print(f'{accuracy_val}%')

print(f'it tooks {finish - start} sec')
plt.plot(loss_history, label='loss')
plt.legend()
plt.draw()

0.9884999990463257%
it tooks 351.1719272136688 sec


In [10]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 28, 28, 1])

In [11]:
L1 = tf.layers.conv2d(x, 32, [3, 3], padding='SAME', activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])

In [12]:
L2 = tf.layers.conv2d(L1, 64, [3, 3], padding='SAME', activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])

In [13]:
L3 = tf.layers.dense(tf.reshape(L2, [-1, 7 * 7 * 64]), units=256, activation=tf.nn.relu)

In [14]:
y_ = tf.layers.dense(L3, units=10, activation=None)

In [15]:
learning_rate = 1e-3
loss_history = []
m = 60000
epoch = 5
mini_batch_size = 128

y = tf.placeholder(tf.float32, [None, 10])

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

start = time()
# <--
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    indices = np.arange(0, m)
    
    for i in range(epoch):
        np.random.shuffle(indices)
        xs = train_images[indices, :]
        ys = train_labels[indices, :]
        
        for j in range(m // mini_batch_size):
            batch_xs = xs[j * mini_batch_size : (j+1) * mini_batch_size, :].reshape(-1, 28, 28, 1)
            batch_ys = ys[j * mini_batch_size : (j+1) * mini_batch_size, :]
            loss_val, _ = sess.run([loss, train_step], feed_dict={x: batch_xs, y: batch_ys})
            loss_history.append(loss_val)
            
    saver = tf.train.Saver()
    saver.save(sess, './models/mnist_cnn')
    # -->    
    finish = time()
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_val = sess.run(accuracy, feed_dict={x: test_images.reshape(-1, 28, 28, 1), y: test_labels})
    print(f'{accuracy_val}%')

print(f'it tooks {finish - start} sec')
plt.plot(loss_history, label='loss')
plt.legend()
plt.draw()

0.9858999848365784%
it tooks 398.36351895332336 sec
